# Introduction to SQL

_SQL_ stands for 'structured query language.' For this ndemonstration we will be using an iPython notebook SQL library developed by Caterine Devlin and others at [https://github.com/catherinedevlin/ipython-sql](https://github.com/catherinedevlin/ipython-sql)

Note that in order to run SQL commands within a Jupyter Notebooks, code blocks need to begin with a 'magic' function:

```
%sql
```

for inline SQL or

```
%%sql
```

for multiple lines of SQL in a code block.

This is a minor addition that is not needed within a standard SQL database or interface, but we like this option because it's notebook friendly and the SQL syntax is otherwise the same.

It may be necessary to install the library:

In [1]:
#!pip install ipython-sql # https://github.com/catherinedevlin/ipython-sql

### Load the extension and connect to a database

In this case we're using SQLite. As a file-based database, this means we don't need to configure a connection to a server.

In [2]:
%load_ext sql
%sql sqlite://

'Connected: None@None'

### Getting Info

Not using a GUI for this demo makes some things more difficult. In particular, database GUIs for SQL and SQLite make it easy to find out information about the database and database objects. But even without a GUI we often need to know things about the database including: 

* the number and names of tables
* structural information and schema

To do this in a notebook we can use SQLite PRAGMA statements. These commands provide control over and information about the SQLite environment. The base syntax needed for using PRAGMA commands here is

```
%sql pragma [command]
```

For example:

```
%sql pragma database_list;
%sql pragma stats;
%sql pragma table_info([table name]);
```

Those three should satisfy most of our requirements today. More information about PRAGMA statements is available in the SQLite documentation [https://www.sqlite.org/pragma.html](https://www.sqlite.org/pragma.html).

### Create a simple table and do stuff

Note that the **IF EXISTS** and **IF NOT EXISTS** flags in the commands below are optional. We use them here because we want a clean, new table each time we run the code block. Without these flags we are more likely to get an error or generate duplicate data.

In [3]:
%%sql
DROP TABLE IF EXISTS donuts;
CREATE TABLE IF NOT EXISTS donuts (doNum, dough, glaze, filling);
INSERT INTO donuts VALUES (1, 'cake', 'maple', 'none');
INSERT INTO donuts VALUES (2, 'yeast', 'sugar', 'none');

Done.
Done.
1 rows affected.
1 rows affected.


[]

Check the status of the database and get some info about it and our table using PRAGMA statements.

In [4]:
%sql PRAGMA DATABASE_LIST;

Done.


seq,name,file
0,main,


In [5]:
%sql PRAGMA STATS;

Done.


table,index,width,height
donuts,None,43,200
sqlite_master,None,65,200


Note that the output above references a system generated table, *sqlite_master* in addition to the table _donuts_ that we just created. This table provides metadata and gives us another way to access info about our tables:

In [6]:
%sql SELECT * from sqlite_master;

Done.


type,name,tbl_name,rootpage,sql
table,donuts,donuts,2,"CREATE TABLE donuts (doNum, dough, glaze, filling)"


In either case, once we know the name of a table we can get more detailed info about it:

In [7]:
%sql PRAGMA TABLE_INFO(donuts);

Done.


cid,name,type,notnull,dflt_value,pk
0,doNum,,0,None,0
1,dough,,0,None,0
2,glaze,,0,None,0
3,filling,,0,None,0


### The SELECT statement

In addition to not being very interesting, our _donuts_ table is poorly structured. There is nothing to prevent duplicate data, enforce non-null or data type requirements, etc. Before going much farther, we want to design a better table. But while we've so far been able to get into ABOUT the table, we haven't yet gotten info FROM the table. Doing so requires use of a SELECT statement. 

The basic syntax is straightforward:

```
SELECT result_column_1, result_column_2 FROM table
```

where a comma separated list of functions or column names are used in place of 'result_column_1, result_column_2' above, and 'table' is replaced with the name of the table we want to view.

Often an asterisk is used as a shortcut for all columns.

In [8]:
%sql SELECT * FROM donuts

Done.


doNum,dough,glaze,filling
1,cake,maple,none
2,yeast,sugar,none


Keep in mind that the SELECT statement refers to columns, not rows. If we want to limit the number of columns returned, we do that accordingly:

```
SELECT glaze FROM donuts
```

which will return just the _glaze_ column and all rows in the table. Limiting the number of rows returned can be done various ways, notably by using a WHERE clause as detailed below.

In [9]:
%sql SELECT glaze FROM donuts

Done.


glaze
maple
sugar


In [10]:
%%sql
INSERT INTO donuts VALUES (3, 'yeast', 'maple', 'boston creme');
INSERT INTO donuts VALUES (4, 'yeast', 'chocolate', 'none');
INSERT INTO donuts VALUES (5, 'cruller', 'vanilla', 'none');
INSERT INTO donuts VALUES (6, 'yeast', 'chocolate ', 'boston creme');

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [11]:
%sql SELECT * FROM donuts

Done.


doNum,dough,glaze,filling
1,cake,maple,none
2,yeast,sugar,none
3,yeast,maple,boston creme
4,yeast,chocolate,none
5,cruller,vanilla,none
6,yeast,chocolate,boston creme


### A closer look at the CREATE statement

As noted, our _donuts_ table is structurally poor. There's nothing to stop us from documenting the same donut twice, omitting information about what kind of dough or glaze is used, etc. Imagine the things that could go wrong!

Earlier we talked about constraints to enforce integrity checks on tables and data. In the next block, we re-create our _donuts_ table with some uniqueness, data typing, non-null, and check constraints.

In [12]:
%%sql
DROP TABLE IF EXISTS donuts;
CREATE TABLE IF NOT EXISTS donuts ('doNum' INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE, 
                                   'dough' TEXT NOT NULL CHECK(dough="cake" OR dough="yeast"),
                                   'glaze' TEXT NOT NULL,
                                   'filling' TEXT);

Done.
Done.


[]

If we inspect the table using PRAGMA, we get a different result from before.

In [13]:
%sql PRAGMA TABLE_INFO(donuts);

Done.


cid,name,type,notnull,dflt_value,pk
0,doNum,INTEGER,1,None,1
1,dough,TEXT,1,None,0
2,glaze,TEXT,1,None,0
3,filling,TEXT,0,None,0


### A closer look at the INSERT statement

Any data entered into _donuts_ now needs to satsify the constraints. Note that this time around we are also specifying the columns.

In [14]:
%%sql
INSERT INTO donuts ('dough', 'glaze', 'filling') VALUES ('cake', 'maple', 'none');
INSERT INTO donuts ('dough', 'glaze', 'filling') VALUES ('yeast', 'sugar', 'none');
INSERT INTO donuts ('filling', 'glaze', 'dough') VALUES ('none', 'maple', 'yeast');
INSERT INTO donuts ('glaze', 'dough','filling') VALUES ('sugar', 'cake', 'none');

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

Selecting all demonstrates that the value for our 'doNum' primary key is automatically generated.

In [15]:
%sql SELECT * FROM donuts

Done.


doNum,dough,glaze,filling
1,cake,maple,none
2,yeast,sugar,none
3,yeast,maple,none
4,cake,sugar,none


The new constraints make it easy to break:

In [16]:
try:
    %sql INSERT INTO donuts ('dough', 'glaze', 'filling') VALUES ('yeast', 'maple', 'boston creme');
    %sql INSERT INTO donuts ('dough', 'glaze', 'filling') VALUES ('yeast', 'chocolate', 'none');
    %sql INSERT INTO donuts ('dough', 'glaze', 'filling') VALUES ('cruller', 'vanilla', 'none');
    %sql INSERT INTO donuts ('dough', 'glaze', 'filling') VALUES ('yeast', 'chocolate ', 'boston creme');
except Exception as e:
    print(str(e))

1 rows affected.
1 rows affected.
(sqlite3.IntegrityError) CHECK constraint failed: donuts [SQL: "INSERT INTO donuts ('dough', 'glaze', 'filling') VALUES ('cruller', 'vanilla', 'none');"]


Correct the error and try again:

In [17]:
try:
    %sql INSERT INTO donuts ('dough', 'glaze', 'filling') VALUES ('yeast', 'maple', 'boston creme');
    %sql INSERT INTO donuts ('dough', 'glaze') VALUES ('yeast', 'chocolate');
    %sql INSERT INTO donuts ('dough', 'glaze') VALUES ('cake', 'vanilla');
    %sql INSERT INTO donuts ('dough', 'glaze', 'filling') VALUES ('yeast', 'chocolate', ' ');
    %sql INSERT INTO donuts ('dough', 'glaze', 'filling') VALUES ('cake', 'vanilla', ' ');
    %sql INSERT INTO donuts ('dough', 'glaze', 'filling') VALUES ('yeast', 'chocolate ', 'boston creme');
except Exception as e:
    print(str(e))

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


In [18]:
%sql SELECT * FROM donuts

Done.


doNum,dough,glaze,filling
1,cake,maple,none
2,yeast,sugar,none
3,yeast,maple,none
4,cake,sugar,none
5,yeast,maple,boston creme
6,yeast,chocolate,none
7,yeast,maple,boston creme
8,yeast,chocolate,None
9,cake,vanilla,None
10,yeast,chocolate,


### Modifying rows with UPDATE

Our table still has some problems. In particular, anything goes in the _filling_ column. That might be fine for donuts, but not for _donuts_!

The ideal solution is to alter the table definition to create some check or constraint on _filling_. The process is a bit convulted in SQLite, so for today we will merely update rows.

In [19]:
%%sql
UPDATE donuts
SET filling = 'none'
WHERE filling IS NULL;

2 rows affected.


[]

In [20]:
%sql SELECT * FROM donuts

Done.


doNum,dough,glaze,filling
1,cake,maple,none
2,yeast,sugar,none
3,yeast,maple,none
4,cake,sugar,none
5,yeast,maple,boston creme
6,yeast,chocolate,none
7,yeast,maple,boston creme
8,yeast,chocolate,none
9,cake,vanilla,none
10,yeast,chocolate,


In [21]:
%%sql
UPDATE donuts
SET filling = 'none'
WHERE filling = ' ';

2 rows affected.


[]

In [22]:
%sql SELECT * FROM donuts

Done.


doNum,dough,glaze,filling
1,cake,maple,none
2,yeast,sugar,none
3,yeast,maple,none
4,cake,sugar,none
5,yeast,maple,boston creme
6,yeast,chocolate,none
7,yeast,maple,boston creme
8,yeast,chocolate,none
9,cake,vanilla,none
10,yeast,chocolate,none


### Modifying rows with the DELETE statement

It is also possible to remove rows.

In [23]:
%%sql
DELETE FROM donuts
WHERE glaze = 'maple';

4 rows affected.


[]

In [24]:
%sql SELECT * FROM donuts

Done.


doNum,dough,glaze,filling
2,yeast,sugar,none
4,cake,sugar,none
6,yeast,chocolate,none
8,yeast,chocolate,none
9,cake,vanilla,none
10,yeast,chocolate,none
11,cake,vanilla,none
12,yeast,chocolate,boston creme


### Filtering rows with WHERE

The WHERE clause is used to filter the execution or output of a command. We used it above to set a condition for updating and deleting rows.

Especially with large and complex datasets, WHERE provides the capability to filter or subset the output of a SELECT statement as well.

In [25]:
%%sql
SELECT *
FROM donuts
WHERE dough = 'yeast';

Done.


doNum,dough,glaze,filling
2,yeast,sugar,none
6,yeast,chocolate,none
8,yeast,chocolate,none
10,yeast,chocolate,none
12,yeast,chocolate,boston creme


The expression which follows the WHERE clause can include multiple conditions.

In [26]:
%%sql
SELECT *
FROM donuts
WHERE dough = 'yeast' AND glaze = 'sugar';

Done.


doNum,dough,glaze,filling
2,yeast,sugar,none


In [27]:
%%sql
SELECT *
FROM donuts
WHERE dough = 'cake' OR filling = 'boston creme';

Done.


doNum,dough,glaze,filling
4,cake,sugar,none
9,cake,vanilla,none
11,cake,vanilla,none
12,yeast,chocolate,boston creme


Use parenthesis to group conditions:

In [28]:
%%sql
SELECT *
FROM donuts
WHERE dough = 'yeast' OR (dough = 'cake' AND glaze = 'sugar');

Done.


doNum,dough,glaze,filling
2,yeast,sugar,none
4,cake,sugar,none
6,yeast,chocolate,none
8,yeast,chocolate,none
10,yeast,chocolate,none
12,yeast,chocolate,boston creme


### GROUP BY and ORDER BY

We may not need to see information about every single donut in the database. If for example we're only interested in knowing how many combinations of cake, glaze, and filling we've got, we can group results using the GROUP BY clause. We can also use ORDER BY to sort the output.

In [29]:
%%sql
SELECT dough, glaze, filling
FROM donuts
GROUP BY dough, glaze
ORDER BY dough, filling;

Done.


dough,glaze,filling
cake,sugar,none
cake,vanilla,none
yeast,chocolate,boston creme
yeast,chocolate,none
yeast,sugar,none


If we need a count of donuts available in each combination, we can use the _count()_ function.

In [30]:
%%sql
SELECT dough, glaze, filling, count(dough) as count
FROM donuts
GROUP BY dough, glaze
ORDER BY dough, filling;

Done.


dough,glaze,filling,count
cake,sugar,none,1
cake,vanilla,none,2
yeast,chocolate,boston creme,1
yeast,chocolate,none,3
yeast,sugar,none,1
